ResNet18, 34, 50, VIP, MVP, DINO


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import zarr

import torch
import numpy as np
import matplotlib.pyplot as plt

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device:", device)

In [ ]:
import torchvision.transforms as transforms
from r3m import load_r3m
import mvp
from vip import load_vip

In [3]:
from src.models.encoders import models

**All Files Loading Info**


In [ ]:
# NOTE: Change this to wherever the project is located in your google drive
project_root = Path(".").resolve().parent
# project_root = Path("/content/drive/MyDrive/17 – Research/sim-2-real-representation-learning") # Lars
data_path = project_root / "data" / "processed"
models_path = project_root / "models"
embeddings_path = project_root / "embeddings"

print("Project root:", project_root)
print("Data path:", data_path)
print("Models path:", models_path)
print("Embeddings path:", embeddings_path)

In [ ]:
# Load the data
sim1 = zarr.open(data_path / "one_leg_low_sim.zarr", mode="r")
sim2 = zarr.open(data_path / "one_leg_med_sim.zarr", mode="r")
real = zarr.open(data_path / "one_leg_low_real.zarr", mode="r")
sim1_imgs = sim1["color_image2"]
sim2_imgs = sim2["color_image2"]
real_imgs = real["color_image2"]
sim1_labels = sim1["action/pos"]
sim2_labels = sim2["action/pos"]
real_labels = real["action/pos"]


print(f"Loaded {len(sim1['episode_ends'])} trajectories containing {sim1_imgs.shape[0]} frames")
print(f"Loaded {len(sim2['episode_ends'])} trajectories containing {sim2_imgs.shape[0]} frames")
print(f"Loaded {len(real['episode_ends'])} trajectories containing {real_imgs.shape[0]} frames")

In [ ]:
# Sample 8 images from each dataset
sim1_indices = np.random.choice(sim1_imgs.shape[0], size=8, replace=False)
sim2_indices = np.random.choice(sim2_imgs.shape[0], size=8, replace=False)
real_indices = np.random.choice(real_imgs.shape[0], size=8, replace=False)

# Create a figure and axes
fig, axes = plt.subplots(2, 8, figsize=(20, 5))

# # Display the sampled images
# for i, idx in enumerate(sim1_indices):
#     axes[0, i].imshow(sim1_imgs[idx])
#     axes[0, i].axis("off")

for i, idx in enumerate(sim2_indices):
    axes[1 - 1, i].imshow(sim2_imgs[idx])
    axes[1 - 1, i].axis("off")

for i, idx in enumerate(real_indices):
    axes[2 - 1, i].imshow(real_imgs[idx])
    axes[2 - 1, i].axis("off")

plt.tight_layout()
plt.show()

**Load Models**


In [ ]:
# Choose the model to use
selected_model_name = "ResNet18"  # Change this to the desired model's name

# Load the selected model
model = models[selected_model_name]().cuda()

model.eval()

### Example of loading our own custom weights

I've trained some models on the data that we've visualized above (the real and the nicely rendered sets). Here, I'll show an example of how we can load up our own weights, and I'd be really curious to see how these embeddings might be different to the pre-trained encoders.


In [ ]:
model_names = [
    "confusion_chkpt_999.pt",
    "naive_chkpt_999.pt",
    "upwt_chkpt_999.pt",
]
# Initialize the model to load the weights into
r3m = load_r3m("resnet18").cuda().eval().module

# Read in the state dict from file
wt_path = models_path / "cotraining" / model_names[0]
state_dict = torch.load(wt_path)

r3m.state_dict().keys()

# Load the weights into the model, and we're ready to use it to predict embeddings
r3m.load_state_dict(state_dict)

**Output Dimensions**


In [7]:
def output():
    if selected_model_name == "MVP" or selected_model_name == "VIP":
        random_input = torch.rand((1, 3, 224, 224)).to(device)  # Move to the correct device
    else:
        random_input = (
            torch.randint(0, 255, size=(1, 3, 224, 224)).float().to(device) / 255.0
        )  # Convert to float and move to device

    # Pass the random data through the encoder
    with torch.no_grad():
        output = model(random_input)

    return output


model_output_dim = output()

**TSNE**


In [ ]:
from src.models.tsne import plot_tsne

file_path = embeddings_path / "encoders/dinov2.npz"

# Sample a subset of the data
n_samples = 1000
dataset = np.load(file_path)


indices = np.random.choice(len(dataset["embeddings"]), n_samples, replace=False)

embeddings = dataset["embeddings"][indices]
domain_labels = dataset["dataset_flag"][indices]

# Plot t-SNE
plot_tsne(embeddings, domain_labels)

**IMAGE FINDER AND Grad-CAM**


In [ ]:
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

model = model
model.eval()

target_layer = model.layer4[-1]

# Initialize Grad-CAM
cam = GradCAM(model=model, target_layers=[target_layer], use_cuda=torch.cuda.is_available())

# Select one image from the sim1 dataset
image_index = 0  # You can change this to select different images
input_image = sim1_images[image_index]

# Preprocess the image (assuming normalization is needed for the encoder)
# Define your normalization parameters based on how the model was trained
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
preprocessed_img = preprocess_image(input_image.cpu().numpy(), mean=mean, std=std)

target_class = 0
grayscale_cam = cam(input_tensor=preprocessed_img, targets=[ClassifierOutputTarget(target_class)])

input_image_np = input_image.cpu().numpy().transpose(1, 2, 0)
input_image_np = (input_image_np - np.min(input_image_np)) / (
    np.max(input_image_np) - np.min(input_image_np)
)


heatmap_image = show_cam_on_image(input_image_np, grayscale_cam, use_rgb=True)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(input_image_np)
plt.title("Original Image")
plt.subplot(1, 2, 2)
plt.imshow(heatmap_image)
plt.title("Grad-CAM Heatmap")
plt.show()

**Domain Probing**


In [ ]:
from src.models.domainprobe import train_domain_probe

file_path = embeddings_path / "encoders/dinov2.npz"

train_domain_probe(file_path, epochs=30, batch_size=200, lr=0.001)

**Domain Labels Alternative**


In [ ]:
from src.models.domain_labels_r2 import domain_r_squared
from src.models.loadsplit import load_and_use_existing_split

file_path = "/home/ubuntu/semrep/embeddings/encoders/dinov2.npz"


train_data, val_data = load_and_use_existing_split(file_path)

dataset = np.load(file_path)
embeddings = dataset["embeddings"]

domain_r_squared(embeddings, dataset["dataset_flag"])